# Federal Polling Average

This is the polling averaging model behind the [cdnpo.li](https://cdnpo.li) website.

In [55]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import BayesianRidge

In [56]:
df2008 = pd.read_csv("/Users/erik/Documents/cdnpoli-src/data/processed/2008_Federal.csv")
df2011 = pd.read_csv("/Users/erik/Documents/cdnpoli-src/data/processed/2011_Federal.csv")
df2015 = pd.read_csv("/Users/erik/Documents/cdnpoli-src/data/processed/2015_Federal.csv")
df2019 = pd.read_csv("/Users/erik/Documents/cdnpoli-src/data/processed/2019_Federal.csv")
df2019['Green'] = df2019['Green'].ffill()

### Approach
We use the expontential weighted moving average to compute the 

In [57]:
df2019['alphas'] = (np.log(df2019['Sample'].astype(float)) / np.log(6)) / 30
df2015['alphas'] = (np.log(df2015['Sample'].astype(float)) / np.log(6)) / 30
print(df2019['alphas'].median())
def ewm(arr, alphas):
    """
    Calculate the EMA of an array arr
    :param arr: numpy array of floats
    :param alpha: float between 0 and 1
    :return: numpy array of floats
    """
    # initialise ewm_arr
    ewm_arr = np.zeros_like(arr)
    ewm_arr[0] = arr[0]
    for t in range(1,arr.shape[0]):
        ewm_arr[t] = alphas[t]*arr[t] + (1 - alphas[t-1])*ewm_arr[t-1]
    return ewm_arr


0.12850972089384688


In [58]:
def apply_ewm(df):
    df['EWM_Liberal'] = ewm(df['Liberal'], df['alphas'])
    df['EWM_Conservative'] = ewm(df['Conservative'], df['alphas'])
    df['EWM_NDP'] = ewm(df['NDP'], df['alphas'])
    df['EWM_Green'] = ewm(df['Green'], df['alphas'])
    df['EWM_BQ'] = ewm(df['Bloc QC'], df['alphas'])
    return df

def evaluate_ewm(df19, df15):
    df19 = apply_ewm(df19)
    df15 = apply_ewm(df15)
    print(df19[['EWM_Liberal', 'EWM_Conservative', 'EWM_NDP', 'EWM_Green', 'EWM_BQ']].tail())
    print("Compared to 2015")
    print(df15[['EWM_Liberal', 'EWM_Conservative', 'EWM_NDP', 'EWM_Green', 'EWM_BQ']].tail())
    return df15[['EWM_Liberal', 'EWM_Conservative', 'EWM_NDP', 'EWM_Green', 'EWM_BQ']].tail(1)

In [59]:
evaluate_ewm(df2019, df2015)

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.004685         34.877977  15.493625   9.361236  4.281002
306    30.065835         34.982958  15.360745   9.539841  4.252138
307    29.976897         34.964165  15.385321   9.741187  4.298142
308    31.387429         35.726354  15.981794  10.373212  4.264797
309    30.147305         34.388961  15.492025  10.143181  4.136525
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    36.475697         31.428351  22.634243   4.565095  4.573819
359    35.881739         31.335434  22.226604   4.642995  4.650449
360    36.049868         31.572887  22.077663   4.802637  4.709312
361    36.288814         31.105761  21.619596   4.521451  4.844647
362    36.273756         30.717860  21.163631   4.485785  4.877464


,EWM_Liberal,EWM_Conservative,EWM_NDP,EWM_Green,EWM_BQ
362,36.273756,30.71786,21.163631,4.485785,4.877464


So the actual results we should get from 2015 should look something like 
39.47 - 31.89 - 19.71 - 3.45 - 4.66


With log base 6 of sample / 30, we miss low on Liberal, low on conservative, high on rest.

In [60]:
df2019['alphas'] = 0.2
df2015['alphas'] = 0.35
print(df2019['alphas'].median())
evaluate_ewm(df2019, df2015)
a = 0.01
while a < 1.0:
    print(a)
    df2015['alphas'] = a
    evaluate_ewm(df2019, df2015)
    a += 0.01

0.2
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    36.809584         31.195334  22.132316   4.334830  4.450341
359    35.931230         31.686967  21.736005   4.707640  4.782722
360    35.885299         31.761529  21.268404   5.019966  4.823769
361    37.325444         31.144994  20.824462   4.312978  5.235450
362    37.946539         30.919246  20.430900   4.413436  5.328042
0.01
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    35.911687         31.221101  22.698553   4.616608  4.556226
359    35.750519         31.358991  22.528697   4.694947  4.640604
360    35.755467         31.413092  22.315828   4.785452  4.666543
361    36.179920         31.271783  22.084245   4.606907  4.799889
362    36.471928         31.194605  21.845820   4.606217  4.869900
0.10999999999999999
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    36.443575         31.213975  22.331698   4.446832  4.528193
359    36.014860         31.491180  22.065358   4.637466  4.702555
360    35.971888         31.572944  21.732287   4.829973  4.742044
361    36.777510         31.258355  21.385829   4.463978  4.993635
362    37.242008         31.106684  21.048663   4.491182  5.094908
0.21000000000000005
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    36.697227         31.210716  22.176749   4.364797  4.481695
359    35.978059         31.627501  21.823724   4.675358  4.757186
360    35.924641         31.709251  21.396607   4.952751  4.800030
361    37.147249         31.196475  20.977625   4.366926  5.160021
362    37.733074         30.987533  20.594337   4.436848  5.262015
0.3100000000000001
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         3

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    36.919627         31.175375  22.101172   4.307963  4.415188
359    35.871776         31.745225  21.660703   4.744778  4.809113
360    35.843066         31.807135  21.156422   5.086867  4.845468
361    37.505839         31.084281  20.693853   4.252120  5.307281
362    38.143504         30.850569  20.296312   4.391272  5.384368
0.4100000000000002
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         3

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    37.136931         31.129034  22.062654   4.259381  4.339093
359    35.718466         31.864517  21.531327   4.829690  4.869546
360    35.759233         31.882259  20.965663   5.214845  4.884773
361    37.879616         30.941129  20.482832   4.107423  5.442387
362    38.489808         30.720565  20.091416   4.353711  5.471193
0.5100000000000002
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         3

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    37.350549         31.078778  22.040690   4.214227  4.262551
359    35.520220         31.991511  21.416276   4.925691  4.945020
360    35.688088         31.936605  20.806510   5.330276  4.918008
361    38.275235         30.774642  20.322604   3.932111  5.567203
362    38.770094         30.609857  19.949042   4.332844  5.526881
0.6100000000000003
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         3

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    37.555791         31.029513  22.026615   4.168926  4.190467
359    35.276737         32.128854  21.307985   5.030678  5.037140
360    35.643021         31.968656  20.672395   5.429203  4.941142
361    38.692906         30.590597  20.201719   3.728761  5.682343
362    38.977872         30.527179  19.850516   4.338628  5.554703
0.7100000000000004
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         3

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    37.743011         30.990492  22.016690   4.120001  4.123749
359    34.988602         32.278098  21.203338   5.144000  5.144750
360    35.637720         31.975620  20.560668   5.508800  4.948950
361    39.127544         30.395124  20.112134   3.501760  5.789790
362    39.105509         30.479025  19.782427   4.380352  5.557958
0.8100000000000005
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         3

     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         35.622045  15.653814  10.153538  4.299727
307    30.301281         35.457636  15.643051  10.422831  4.379782
308    30.841024         34.766109  15.714441  10.738265  4.103825
309    30.572820         34.672887  15.851553  10.890612  4.163060
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    37.897369         30.975342  22.008641   4.064540  4.060848
359    34.659737         32.437534  21.100864   5.266454  5.266085
360    35.685974         31.953753  20.470086   5.566645  4.936608
361    39.568597         30.195375  20.047009   3.256665  5.893661
362    39.146860         30.469538  19.734701   4.465666  5.539366
0.9100000000000006
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.570751         35.552556  16.017267   9.916923  4.374659
306    30.576601         3

Generally, the most coherent results seem to be yielded around alpha = 0.18

In [61]:
df2019['alphas'] = 0.18
df2015['alphas'] = 0.18
evaluate_ewm(df2019, df2015)


     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
305    30.631002         35.612553  15.996376   9.855178  4.388785
306    30.625422         35.664293  15.673029  10.079246  4.318803
307    30.368846         35.508720  15.659884  10.334981  4.387419
308    30.842454         34.877151  15.721104  10.634685  4.137683
309    30.600812         34.773264  15.843306  10.790442  4.184900
Compared to 2015
     EWM_Liberal  EWM_Conservative    EWM_NDP  EWM_Green    EWM_BQ
358    36.378998         31.209566  22.378465   4.470414  4.534821
359    36.004778         31.459844  22.130341   4.637739  4.690553
360    35.967918         31.539072  21.818880   4.810946  4.728254
361    36.693693         31.262039  21.491481   4.484976  4.957168
362    37.126828         31.124872  21.169015   4.505680  5.054878


,EWM_Liberal,EWM_Conservative,EWM_NDP,EWM_Green,EWM_BQ
362,37.126828,31.124872,21.169015,4.50568,5.054878
